In [ ]:
import pandas as pd
import numpy as np
import time
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

Here is a link to the data: [Black Friday](https://www.kaggle.com/mehdidag/black-friday). Here is the original posting of the data set from [Analytics Vidhya](https://datahack.analyticsvidhya.com/contest/black-friday/#problem_statement). The data is from Indian department stores and is posted on the website for use in a contest. The contest is to predict purchases and that will be one of the business questions I analyze in this notebook. 

[Here is a Medium Blogpost](https://medium.com/@michaelrobertreinhard/men-are-doing-the-shopping-depending-on-the-product-63393536595c) based on this analysis. 

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("BlackFriday.csv.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
df = pd.read_csv('BlackFriday.csv', sep=',')

In [ ]:
df.head()

In [ ]:
df.shape

## Data Wrangling
As this data was taken from a contest it is, in the main, ready for analysis, but there are a few things that need to be done to it. The missing values have to be dealt with and some variables need their type adjusted.
### Missing Values
First I look for missing values. 

In [ ]:
print(df.isna().sum())

The missing values appear to be in the Product Categories 2 and 3. My guess is that these are the subcategories of product 1 so when they are missing I will just the product category 1.  

In [ ]:
#substitue Product Category 1 for missing data in 2 and 3
df['Product_Category_2'] = df['Product_Category_2'].fillna(
    df['Product_Category_1']) 
df['Product_Category_3'] = df['Product_Category_3'].fillna(
    df['Product_Category_1']) 

### Mis-Typed data
There are some data that have the wrong type. For instance age is presently coded as a string variable becuase of the categories, like '0-17', '55+', it was coded into. 

In [ ]:
df['Age'] = df['Age'].replace(
    {'0-17':int(15), 
     '55+':int(65),
     '18-25':int(22),
     '26-35':int(31),
     '36-45':int(41),
     '46-50':int(48),
     '51-55':int(53)})

The same is true for the number of years spent in current city, though this variable only codes up to a maximum of 4 years. 

In [ ]:
df['Stay_In_Current_City_Years'].head()

In [ ]:
#turn into an integer 
df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].replace(
    {'0':int(0), 
     '1':int(1), 
     '2':int(2), 
     '3':int(3), 
     '4+':int(4)})

## Answering Business Questions

### Are there differences between groups?
This is marketing data and the main use it can be put to is singling out groups for special marketing efforts. Therefore, the main group of hypotheses to test are whether there are differences in purchases among various groups. 

In [ ]:
df.groupby('Occupation')['Purchase'].mean()

That looks like some very small differences, too small to be statistically significant. But to be sure lets apply a t test to these differences. I first obtain the standart error for the group difference and then test to flag any group differences that are more than two standard deviations from the mean for the entire data set. 

In [ ]:
mean = df['Purchase'].mean()
standard_deviation = df['Purchase'].std()

In [ ]:
sig_occ_groups = []
for occ_group in df.groupby('Occupation')['Purchase'].mean():
#     print(abs(mean - 2*standard_deviation) < abs(mean - occ_group))
    if abs(mean - 2*standard_deviation) < abs(mean - occ_group):
        sig_occ_groups.append(occ_group)

In [ ]:
print(sig_occ_groups)

So the first business hypothesis, that purchases differ significantly by occupation, is shown to be false.  

But this exercise points to a procedure that could be usefully automated. We seek to find any groups that differ in a statistically significant way from the mean of the data set in purchases. Why not make the above procedure a functtion that could be applied to all of the groups in the data set? That is what I am going to do. The function will take a category of consumers, group the data set by them, and then test to see if the groups differ from the data set mean in a statistically significant way.  

I will assume the data set mean and standard deviations to be globally available to the data set in this analysis.

In [ ]:
def group_diff(df, group, mean, std_dev):
    sig_groups = []
    for index, group_mean in enumerate(df.groupby(group)['Purchase'].mean()):
        if abs(mean - 2*std_dev) < abs(mean - group_mean):
            sig_groups.append(index)
    return sig_groups
        
occupation_groups = group_diff(df=df, group='Gender', 
                               mean=df['Purchase'].mean(), 
                               std_dev=df['Purchase'].std())
print(occupation_groups)

In [ ]:
gender_groups = group_diff(df=df, group='Gender', 
                           mean=df['Purchase'].mean(), 
                           std_dev=df['Purchase'].std())
print(gender_groups)

In [ ]:
city_groups = group_diff(df=df, group='City_Category', 
                         mean=df['Purchase'].mean(), 
                         std_dev=df['Purchase'].std())
print(city_groups)

In [ ]:
marital_groups = group_diff(df=df, group='Marital_Status', 
                         mean=df['Purchase'].mean(), 
                         std_dev=df['Purchase'].std())
print(marital_groups)

In [ ]:
#We can give groupby lists allowing up to test subgroups
gender_maritalStatus_groups = group_diff(df=df, group=['Gender', 'Marital_Status'], 
                                         mean=df['Purchase'].mean(), 
                                         std_dev=df['Purchase'].std())

print(gender_maritalStatus_groups)

In [ ]:
Age_groups = group_diff(df=df, group='Age', 
                        mean=df['Purchase'].mean(), 
                        std_dev=df['Purchase'].std())
print(Age_groups)

So this gives a fuller answer to our first business question, are there any significant differences between groups in their purchases? The answer is no, at least for this data set. In all the demographic groups we have the differences in mean purchases by group are not statistically significant. 

### Product Categories
But if there are no statistically significant differences between groups in the average amount of purchases they make, does that hold true for every product category? Here I will only ask the question for one demographic distinction: gender.

In [ ]:
male_means = df[df['Gender']=='M'].groupby(['Product_Category_1', 
                                            'Gender'])['Purchase'].count()\
                                            /len(df[df['Gender']=='M'])
    
female_means = df[df['Gender']=='F'].groupby(['Product_Category_1', 
                                              'Gender'])['Purchase'].count()\
                                              /len(df[df['Gender']=='F'])

male_stderr = df[df['Gender']=='M'].groupby(['Product_Category_1', 
                                             'Gender'])['Purchase'].std()\
                                             /len(df[df['Gender']=='M'])
    
female_stderr = df[df['Gender']=='F'].groupby(['Product_Category_1', 
                                               'Gender'])['Purchase'].std()\
                                               /len(df[df['Gender']=='F'])

In [ ]:
from matplotlib.pyplot import figure
from matplotlib.ticker import MaxNLocator

plt.clf()

fig, ax = plt.subplots()

ind = df['Product_Category_1'].unique()
ind = np.array(sorted(ind))

fig.set_size_inches(14, 8)

width = 0.40  

p1 = plt.bar(ind, male_means, width, yerr=male_stderr, label='Male')
p2 = plt.bar(ind + width, female_means, width, yerr=female_stderr, label='Female')

ax.set_xlabel('Product Catogories')
ax.set_ylabel('Average Number of Purchases by Gender')
plt.title('Purchases by Product Category and Gender')
ax.set_xticks(ind + width/2)
ax.set_xticklabels(('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18'))

plt.legend();

Now we can see the differences between the purchases of men and women by product category. We can see by the overlap of the error bars that the differences are not statistically significant in all but three categories, the three with the greatest differences in the means for men and women. Lets adjust our chart to emphasize those three categories of product purchases.

In [ ]:
diff = male_means.values - female_means.values

In [ ]:
orig_dict = {}
for i,j in zip([x for x in range(1,19)],diff):
    orig_dict[i] = j
print(orig_dict)

In [ ]:
ord_dict = OrderedDict()
for i,j in zip(diff, [x for x in range(1,19)]):
    ord_dict[i] = j
print(ord_dict)

Now I have the values in an ordered dict with their indexes, the category numbers, as thier values. Next, I want to create a new dictionary with those same values as keys, but this time with thier absolute values. 

In [ ]:
abs_val_dict = dict()
for key, value in ord_dict.items():
    abs_val_dict[abs(key)] = value
print(abs_val_dict)

Now I use the absolute values to sort the dictionary values by the absolute value of their keys and I will save this to another data structure.

In [ ]:
category_list = []
for i in sorted(abs_val_dict.keys(),reverse=True):
    category_list.append(abs_val_dict[i])
print(category_list)

Then I use this list of category names to get the original values.

In [ ]:
values = []
for i in category_list:
    values.append(orig_dict[i])

values_arr = np.array(values)
category_arr = np.array(category_list)

In [ ]:
category_arr_str = [str(i) for i in category_arr]

In [ ]:
plt.clf()
plt.cla()

fig = plt.gcf()

fig.set_size_inches(14, 8)

width = 0.70

plt.barh(category_arr_str[::-1], values_arr[::-1], width)
plt.xlabel('Difference in Proportion')
plt.xticks=category_arr_str


plt.ylabel('Product Category')


plt.annotate(xy = [-0.04, 2], s='Women', fontsize=30)
plt.annotate(xy = [0.04, 2], s='Men', fontsize=30)
plt.title('Biggest Differences in Male and Female Spending by Category');

So from this graph it is easy to see that product category 1 is favored by men by a nearly 10% margin, while categories 8 and 5 are favored by women by around 5%. This is something to be taken into account in business decisions. 

## Prep for machine learning

Create the target variable.

In [ ]:
y = df['Purchase']

### Drop unused variables
Now that we have put purchase in the 'y' or target data we have to get rid of it. I experimented with leaving out other variables and so wrote a function that defaults to removing 'purchase', 'User_ID', and 'Product_ID'. I left these out because they are not really general enough to provide an real insights to policy. Predicting how much a person spend from the product they bought is largely an exercise in predicting the price of the product. The same problem presents itself with the User_ID. We can predict the individual's behavior but can't generalise from that without knowing some characteristics that person shares with other. 

In [ ]:
#make a function to drop unused categories
def drop_col(df, use_product_factor=False, category=False):
    '''drop categories that are unused, making User_ID 
       and Product_ID optional and making Product_Category_ID optional'''
    
    if use_product_factor:   
        my_list = ['Purchase', 
                   'User_ID', 
                   'Product_ID']
    else:
        my_list = ['Purchase']
    
    if category:
        my_list.extend(['Product_Category_1', 
                        'Product_Category_2', 
                        'Product_Category_3'])
    
    for col in my_list:
        try:
            df.drop(col, inplace=True, axis=1)
        except:
            'column has already been deleted'

In [ ]:
drop_col(df)

### Prepare list of qualitative variables
The next thing we have to do is get the qualitative variables into a data frame that we can work with. Here I have created a function that defaults to four qualitative variables and includes the option to add product ID and the Product Categories. I have found that the default is the most informative.

In [ ]:
def select_qual_var(df, Product_ID=False, Product_Category=True):
    '''creates list of qualitative variables. Includes product category and
    excludes Product_ID by default'''
    
    my_list = ['Occupation', 'Marital_Status', 'Gender', 'City_Category']
    
    if Product_ID==True:
        my_list.append('Product_ID')
    if Product_Category==True:
        my_list.extend(['Product_Category_1', 'Product_Category_2', 'Product_Category_3'])
    df_qual = df.loc[:, my_list]
    
    return df_qual

In [ ]:
df_qual = select_qual_var(df)

### Get dummies
Now I have to get the dummy variables from the qualitative variables and drop the original qualitative variables. I do this in a for loop. 

In [ ]:
#make dummy variables of the qualitative variables
#and drop the original variable
for var in df_qual.columns:
    df_qual = pd.concat(
                [df_qual.drop(var, axis=1), 
                 pd.get_dummies(df_qual[var], 
                   drop_first=True, 
                   prefix=var, 
                   prefix_sep='_')], 
                 axis=1)

In [ ]:
df_qual.shape

### Get quantitative variables

In [ ]:
df_quant = df.select_dtypes(['float', 'int'])

In [ ]:
df_quant.columns

### Join the two data frames

In [ ]:
df = df_quant.join(df_qual)

### Train test split
I create a function to do this because I wanted to use a sample of the data frame for experimentation. 

In [ ]:
def get_train_test_split(df, y, sample=False):
    '''performs train_test_split with option to 
    create a smaller sample data set of 5000.'''
    
    if sample:
        df = df.sample(n=5000, random_state=42)
        y = y.sample(n=5000, random_state=42)
    
    X = df
    y = y
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_split(df, y)

### Scale the variables

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  

## The Model
I finally present the final model to answer the question how well can we predict purchases. I have left the trial and error for the notebook "Black Friday", included in this repository. There I went through Linear Regression and it regularized variants, lasso and ridge regression, and finally onto Random Forest Regression, the results of which, I present here.  

Using grid search, I manipulated the maximum depth, features, and leaf nodes, as well as the minimum samples to form a leaf and the number of estimators. I present the results of the best model here. 

There was little loss in performance of the model from the training to the test data, with the $R^2$ dropping from 65.20 to 65.07. That means we can explain 65% of the variance in purchases by the model. 





In [ ]:
params={'max_depth': [21], 'max_features': [69], 'max_leaf_nodes': [695],
        'min_samples_leaf': [1], 'n_estimators': [500]}
cv = GridSearchCV(estimator=RandomForestRegressor(random_state=42), 
                  param_grid=params, verbose=1, cv=3, n_jobs=-1)


In [ ]:
t0 = time.time()

results = cv.fit(X_train,y_train)

t1 = time.time()

total = t1 - t0
print(total/60)

In [ ]:
print(results.best_estimator_)
print(results.best_score_)
print(results.best_params_)
y_test_pred = cv.predict(X_test)

print(r2_score(y_test, y_test_pred))
print(np.sqrt(mean_squared_error(y_test, y_test_pred)))